# Analysis

The final stroke of the project involve analysing the networks created:

In [1]:
## import required packages
import os
import pickle as pkl
import pandas as pd
import networkx as nx
import numpy as np
from numpy.random import RandomState
rng = RandomState(781772)
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## graphics:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['seaborn-poster'])

# home-made packages
import network_artist as artist
from infnet_helper import *

In [2]:
lookup_combined_toks = prepare_toks(with_pdf2txt=False)

In [3]:
lookup_poinf = get_lookup_poinf()

In [4]:
# remove individuals without  any publications since they wont appear in the collab net
lookup_poinf.dropna(subset=['pub_ids'], inplace=True)
print(len(lookup_poinf))

224


In [23]:
lookup_poinf.head(3)

,last_name,first_name,personal_url,position,parent,institute,full_name,institute_class,alias,first_pub_year,total_pub,total_pub_1997,total_pub_2012,pub_ids
id,,,,,,,,,,,,,,
d089bb44-7d0b-4b3f-b556-27def993d1d1,albrecht,stefano,http://www.research.ed.ac.uk/portal/en/persons...,lecturer in artificial intelligence,school of informatics,centre for intelligent systems and their appli...,albrecht stefano,1,"albrecht, s. v.|albrecht, s.",2014.0,12,12,12,"{ca133725-ba89-4acc-984b-d54e667270f6, 990f481..."
ecd799fb-4f63-44ae-a078-b009099f2c8c,alex,beatrice,http://www.research.ed.ac.uk/portal/en/persons...,research fellow,school of informatics,institute of language cognition and computation,alex beatrice,2,"alex, b. (ed.)|alex, b.|symington, b.",2012.0,43,43,20,"{6b8df6e9-705a-43be-bdce-8b612e39e71d, 5c90210..."
1997d4be-de27-44bd-ad89-36e6f386225d,alexandru,cristina,http://www.research.ed.ac.uk/portal/en/persons...,research associate,school of informatics,centre for intelligent systems and their appli...,alexandru cristina,1,"alexandru, c.|alexandru, c. a.|alexandru, c-a.",2011.0,9,9,7,"{ec8fd94d-aea9-4320-86f1-2250352f698c, d850fd3..."


In [ ]:
lookup_pub = 

## Homophily - Assortative mixing

### infnet20yr

In [48]:
x = dict((a,b) for (a,b) in
                           zip(lookup_poinf.index.tolist(),
                               lookup_poinf.full_name.tolist()))

In [6]:
# infnet20yr:
g_infnet20yr = nx.Graph()
for poinf_id, attr in lookup_poinf.iterrows():
    g_infnet20yr.add_node(poinf_id)

In [11]:
lookup_poinf.loc['ecd799fb-4f63-44ae-a078-b009099f2c8c']['full_name']

'alex beatrice'

In [20]:
g_infnet20yr.node

NodeView(('d089bb44-7d0b-4b3f-b556-27def993d1d1', 'ecd799fb-4f63-44ae-a078-b009099f2c8c', '1997d4be-de27-44bd-ad89-36e6f386225d', 'ca066cb3-40b1-4721-a12e-edbdb031cf7c', 'c3bee2e3-cad6-4ae1-9d94-d17f8bb39935', 'e4fface3-9781-4bfa-9a40-9340f707cde3', 'da3124d7-44f9-4199-82bf-9e964f422ab9', 'dbb07214-3753-4bba-a630-e1857e7efb09', '2db0a63a-084d-4b6e-ba29-416854572865', '8e51eafb-dd78-468b-8970-ec2349e92305', '71021818-cf31-4cca-9dc9-af9db85e31a9', '91770438-791c-43da-8fcf-3bdd8f2293fd', '9129cbb9-4d56-4bab-8124-1e34a5b01f70', '1a05e95b-6e1a-40d8-8406-d5e62b7c722d', '688a89f2-45fd-42ad-8fcd-08a065521c0a', '570a3e8e-0e1c-4d8e-8cf5-8374bdc9a5c4', '2c3dcad6-2934-40ab-8663-8a41c0597024', 'bd14c9d9-1e8b-4816-b547-d3727e0d8537', 'aa593eb8-2ac7-415c-aec5-738f11b6e795', '0b2fae7b-cf7f-4f8b-a92c-4fa055ff9d63', '350d5a79-901a-4018-9e1a-87114b8dbe0b', 'f804a9ce-5cc1-456b-b652-a91c8e034c68', 'a78a09b5-138e-407e-8c09-34414853eb20', '412b9b6c-a9d5-47d6-81af-018323057f36', '213fbdf4-a67d-4b06-8614-3b864

In [19]:
g = nx.Graph()
g.add_node(1, val{'name':'hl'})

TypeError: add_node() takes 2 positional arguments but 3 were given